In [12]:
%matplotlib inline


SyntaxError: invalid syntax (<ipython-input-12-4b5c722e6181>, line 2)

In [2]:
# Import Dependicies 
import gmaps 
import pandas as pd 
import matplotlib.pyplot as plt 
import json
import requests 
from config import g_key


In [3]:
# Import csv file 

file_to_upload = "Data/WeatherPy_challenge.csv"

weather_data_df = pd.read_csv(file_to_upload)
weather_data_df = weather_data_df.set_index("City_ID")
weather_data_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
City_ID,,,,,,,,,,,
0,Punta Arenas,CL,-53.15,-70.92,42.80,93,90,9.17,light rain,0,0
1,Castro,BR,-24.79,-50.01,62.38,70,0,3.76,clear sky,0,0
2,Butaritari,KI,3.07,172.79,78.96,77,100,11.83,light rain,0,0
3,Manado,ID,1.49,124.85,75.20,94,40,3.20,scattered clouds,0,0
4,Shelburne,CA,44.08,-80.20,53.60,81,90,11.41,overcast clouds,0,0


In [6]:
# Obtain inputs from user for Maximum Temperature, Minimum Temperature, and raining/snowing preferences 
min_temp = float(input("What is your minimum temperature (F) preference?"))
max_temp = float(input("What is your maximum temperature (F) preference?"))
rain_status = input("Do you want it to be raining? (yes/no)")
snow_status = input("Do you want it to be snowing? (yes/no)")



What is your minimum temperature (F) preference?60
What is your maximum temperature (F) preference?90
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [8]:
# Checking data types to ensure gmaps compatibility 
weather_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)            int64
Snow (inches)            int64
dtype: object

In [9]:
# Filtering data based on above user input conditions 

filtered_df = weather_data_df.loc[(weather_data_df["Max Temp"] >= min_temp) & 
                                  (weather_data_df["Max Temp"] <= max_temp)]
if snow_status == "yes":
    filtered_df = filtered_df.loc[(filtered_df["Rain (inches)"] > 0)]
else:
    filtered_df = filtered_df.loc[(filtered_df["Rain (inches)"] == 0)]

if rain_status == "yes": 
    filtered_df = filtered_df.loc[(filtered_df["Snow (inches)"] > 0)]
else:
    filtered_df = filtered_df.loc[(filtered_df["Snow (inches)"] == 0)]

filtered_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
City_ID,,,,,,,,,,,
1,Castro,BR,-24.79,-50.01,62.38,70,0,3.76,clear sky,0,0
2,Butaritari,KI,3.07,172.79,78.96,77,100,11.83,light rain,0,0
3,Manado,ID,1.49,124.85,75.20,94,40,3.20,scattered clouds,0,0
5,Jacqueville,CI,5.21,-4.41,80.60,83,20,5.82,few clouds,0,0
6,Rikitea,PF,-23.12,-134.97,74.77,81,13,11.39,few clouds,0,0


In [10]:
vacation_df = filtered_df[["City", "Country", "Max Temp", "Current Description",
                          "Lat", "Lng"]].copy().reset_index()
vacation_df["Hotel Name"] = ""
vacation_df = vacation_df.set_index("City_ID")
vacation_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Castro,BR,62.38,clear sky,-24.79,-50.01,
2,Butaritari,KI,78.96,light rain,3.07,172.79,
3,Manado,ID,75.20,scattered clouds,1.49,124.85,
5,Jacqueville,CI,80.60,few clouds,5.21,-4.41,
6,Rikitea,PF,74.77,few clouds,-23.12,-134.97,
9,Cacu,BR,73.89,broken clouds,-18.56,-51.13,
10,Vaini,TO,73.40,few clouds,-21.20,-175.20,
16,Ahuimanu,US,87.80,few clouds,21.44,-157.84,
19,Ribeira Grande,PT,66.20,broken clouds,38.52,-28.70,


In [11]:
## Find Hotels in the cities:

# Set Parameters to search for a hotel.
params = {"radius":5000,
         "type": "lodging",
         "key": g_key,
         "location": ""}

# Iterate through the DataFrame. 
for index, row in vacation_df.iterrows():
    
    # Get the latitude and longtitude
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    # add the latitude and longitude to location key for the params dict
    params["location"] = f"{lat},{lng}"
    
    # Use base url to search for hotels:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params = params).json()
    try: 
        # Grab the first hotel from the results and store the name. 
        vacation_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print(f"Hotel not found in {city}.... skipping")
        
print("Hotel data gathering completed.")

Hotel not found in Albany.... skipping
Hotel not found in Zemio.... skipping
Hotel not found in Abu Kamal.... skipping
Hotel not found in Luba.... skipping
Hotel not found in Zabol.... skipping
Hotel not found in Palauig.... skipping
Hotel not found in Longonjo.... skipping
Hotel not found in Mitu.... skipping
Hotel not found in Toftir.... skipping
Hotel not found in Niquero.... skipping
Hotel not found in Amapa.... skipping
Hotel not found in Kibala.... skipping
Hotel not found in Ijaki.... skipping
Hotel not found in Dicamay.... skipping
Hotel not found in Lagos.... skipping
Hotel not found in Mtimbira.... skipping
Hotel not found in Polovinnoye.... skipping
Hotel not found in Youkounkoun.... skipping
Hotel not found in Kabo.... skipping
Hotel not found in Lodja.... skipping
Hotel not found in Thinadhoo.... skipping
Hotel not found in Avera.... skipping
Hotel not found in Awjilah.... skipping
Hotel not found in Morgaushi.... skipping


In [13]:
# COnfiguring gmaps using g_key
gmaps.configure(api_key = g_key)


In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [22]:
#Assign the locations to an array of latitude and longitude pairs.
locations = vacation_df[["Lat",  "Lng"]]
# Assign values for layers
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
#Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.25)
# Assign layers:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info) 
# Add layers
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
# Exporting new data frame to excel 

file_name = "Data/WeatherPy_vacation.csv"
vacation_df.to_csv(file_name, index_label = "City_ID")